In [4]:
import cv2 
import pandas as pd 
import numpy as np 
import matplotlib as plt 

### Harris


In [24]:
imgobj = cv2.imread('img/green_face.jpg',)
gray_img = cv2.cvtColor(imgobj, cv2.COLOR_BGR2GRAY)
gray_img = np.float32(gray_img)
corners = cv2.goodFeaturesToTrack(gray_img, maxCorners=50,     qualityLevel=0.20, minDistance=20)
corners = np.float32(corners)

for item in corners:
    x, y = item[0]

    cv2.circle(imgobj, (int(x), int(y)), 50, (255, 0, 0), -1)


cv2.imshow('',imgobj)
 
cv2.waitKey(0)
cv2.destroyAllWindows()

In [89]:
def angle_cos(p0, p1, p2):
    import numpy as np

    d1, d2 = (p0-p1).astype('float'), (p2-p1).astype('float')
    return abs( np.dot(d1, d2) / np.sqrt( np.dot(d1, d1)*np.dot(d2, d2) ) )

In [34]:
imgobj = cv2.imread('img/green_face.jpg',)
gray_img = cv2.cvtColor(imgobj, cv2.COLOR_BGR2GRAY)
gray_img = np.float32(gray_img)
blurred = cv2.GaussianBlur(gray_img, (3, 3), 0)
canny = cv2.Canny(gray_img, 40, 80)
cv2.imshow('',canny)
 
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.5.4-dev) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/canny.cpp:829: error: (-215:Assertion failed) _src.depth() == CV_8U in function 'Canny'


In [90]:
# import the opencv library
import cv2
vid = cv2.VideoCapture(0) 
while(True):   
    # Capture the video frame
    # by frame
    ret, frame = vid.read()



    gray_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_img = np.float32(gray_img)
    blurred = cv2.GaussianBlur(gray_img, (3, 3), 0)
    canny = cv2.Canny(frame, 20, 40)
    kernel = np.ones((3,3), np.uint8)
    dilated = cv2.dilate(canny, kernel, iterations=2)
    (contours, hierarchy) = cv2.findContours(dilated.copy(), cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)




    squares=[]
    for cnt in contours:
                    cnt_len = cv2.arcLength(cnt, True)
                    cnt = cv2.approxPolyDP(cnt, 0.02*cnt_len, True)
                    if len(cnt) == 4  and cv2.isContourConvex(cnt):
                        cnt = cnt.reshape(-1, 2)
                        max_cos = np.max([angle_cos( cnt[i], cnt[(i+1) % 4], cnt[(i+2) % 4] ) for i in range(4)])
                        if max_cos < 0.3:
                            squares.append(cnt)

    cv2.drawContours(frame, squares, -1, (0,255,0), 3)


    # Display the resulting frame
    cv2.imshow('frame',frame)
    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  
# After the loop release the cap object
vid.release()

cv2.destroyAllWindows()

In [87]:
    for c in contours : 
        if (c.shape[0]==4):
            print(c,"hey")

[[[253 652]]

 [[254 651]]

 [[255 652]]

 [[254 653]]] hey
[[[797 710]]

 [[798 709]]

 [[799 710]]

 [[798 711]]] hey
[[[798 708]]

 [[799 707]]

 [[800 708]]

 [[799 709]]] hey
[[[799 707]]

 [[800 706]]

 [[801 707]]

 [[800 708]]] hey
[[[943 702]]

 [[944 701]]

 [[945 702]]

 [[944 703]]] hey
[[[965 691]]

 [[966 690]]

 [[967 691]]

 [[966 692]]] hey
[[[534 688]]

 [[535 687]]

 [[536 688]]

 [[535 689]]] hey
[[[948 687]]

 [[949 686]]

 [[950 687]]

 [[949 688]]] hey
[[[537 678]]

 [[538 677]]

 [[539 678]]

 [[538 679]]] hey
[[[387 677]]

 [[388 676]]

 [[389 677]]

 [[388 678]]] hey
[[[940 671]]

 [[941 670]]

 [[942 671]]

 [[941 672]]] hey
[[[933 670]]

 [[934 669]]

 [[935 670]]

 [[934 671]]] hey
[[[949 662]]

 [[950 661]]

 [[951 662]]

 [[950 663]]] hey
[[[389 659]]

 [[390 658]]

 [[391 659]]

 [[390 660]]] hey
[[[924 646]]

 [[925 645]]

 [[926 646]]

 [[925 647]]] hey
[[[915 645]]

 [[916 644]]

 [[917 645]]

 [[916 646]]] hey
[[[821 644]]

 [[822 643]]

 [[823 644]]

In [78]:
type(contours)


numpy.ndarray

In [80]:



print(len(tuple(tab)),len(contours))
        

        


98 414
